In [1]:
import os
import cv2
import face_recognition
import socket
import io
import numpy as np
from PIL import Image
from PIL import ImageFile

# folder containing images
path = r'C:\Users\Andrew Kim\Desktop\Python Script\Photos'

list_names=[]
list_faces=[]

# find all images in the folder
for im in os.listdir(path):
    # verify the format of the image
    if im[-4:] == '.jpg':
        list_names.append(im[:-4])
        temp_face = face_recognition.load_image_file(path+'\\'+im)
        list_faces.append(face_recognition.face_encodings(temp_face)[0])

In [2]:
list_names

['andrew', 'ben', 'ehsan', 'fay', 'kelby', 'tibor', 'will']

In [3]:
#Takes care of error of image sizes
ImageFile.LOAD_TRUNCATED_IMAGES = True

TCP_IP = '129.8.232.106'
TCP_PORT = 11000

print('Starting to Listen')
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
conn, addr = s.accept()
print('Connection address:', addr)

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
sendData = ''
check = True

while True:
    check = True
    #Get data
    data = conn.recv(1024000)
    pilImage = Image.open(io.BytesIO(data))
    pilImage = pilImage.transpose(Image.FLIP_LEFT_RIGHT)
    #frame = cv2.cvtColor(np.array(pilImage), cv2.COLOR_RGB2BGR)
    frame = cv2.cvtColor(np.array(pilImage), cv2.COLOR_RGB2BGR)

    # Resize frame of video to 1/4 size for faster face recognition processing
    #small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Find all the faces and face encodings in the current frame of video
#     face_locations = face_recognition.face_locations(small_frame)
#     face_encodings = face_recognition.face_encodings(small_frame, face_locations)
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces(list_faces, face_encoding)
        name = "Unknown"

        for i in range(len(match)):
            if match[i]:
                name = list_names[i]

        face_names.append(name)

    count = 0
    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
#         top *= 4
#         right *= 4
#         bottom *= 4
#         left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        
#         if check:
#             sendData = name + ' ' + str(top) + ' ' + str(right) + ' ' + str(bottom) + ' ' + str(left) + '\n'
#             conn.send(str.encode(sendData))
#             check = False
        #print(sendData)
        count += 1
        if count == 2: break
        
    # Display the resulting image
    cv2.imshow('Video', frame)
    cv2.waitKey(1)
    #Find all faces in the image
    
    #User Input
#     if check:
#         conn.send(str.encode('Nothing -100 -100 -100 -100\n'))
    #To close
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
conn.close()

# Release handle to the webcam
cv2.destroyAllWindows()

Starting to Listen
Connection address: ('129.8.194.8', 49859)


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host